In [2]:
import pandas as pd
import requests
import urllib3
import ssl
import io

In [3]:
all_countries = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                 'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                 'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore',
                 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland',
                 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
                 'China', 'Taiwan', 'India', 'Korea', 'Brazil',
                 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
                 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
                 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
                 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']

In [4]:
all_countries_in_oecd = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                         'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                         'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway',
                         'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
                         'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa',
                         'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece',
                         'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia']

In [5]:
def format_oecd_data(df):
    df_formatted = pd.DataFrame(index=df['TIME'].unique())

    for country in df['Country'].unique():
        df_formatted[country] = df[df['Country'] == country]['Value']

    df_formatted.rename(columns={
        'Czech Republic': 'Czechia',
        'Türkiye': 'Turkiye',
        "China (People's Republic of)": 'China',
        'Euro area (19 countries)': 'Eurozone'
    }, inplace=True)

    df_formatted = df_formatted.reindex(columns=all_countries_in_oecd)
    return df_formatted

In [32]:
def check_data_coverage(df):
    available_countries = df.columns[~df.isna().all()]
    print('Available countries:')
    print(available_countries)
    print()

    no_data = df.columns[df.isna().all()]
    print('No data:')
    print(no_data)
    print()

    print('Missing first:')
    for country in df.columns:
        first_valid = df[country].first_valid_index()
        if first_valid != None and (not str(first_valid).startswith('1999')):
            print(country, first_valid)
    print()

    print('Missing last:')
    for country in df.columns:
        last_valid = df[country].last_valid_index()
        if last_valid != None and (not str(last_valid).startswith('2023')):
            print(country, last_valid)
            
    print()
    print('Done')

In [7]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [26]:
countries = 'AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19'

def get_oecd_data(indicator, subject, measure, freq, file_name, 
                  countries_before_subject=False, with_measure=True,
                  filter_measure_on_df=False):
    if freq == 'Q':
        startTime = '1999-Q1'
    else:
        startTime = '1999-01'

    if countries_before_subject:
        countries_and_subject = f'{countries}.{subject}'
    else:
        countries_and_subject = f'{subject}.{countries}'
    
    if with_measure:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{measure}.{freq}/all?startTime={startTime}&contentType=csv'
    else:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{freq}/all?startTime={startTime}&contentType=csv'

    print(url)
    res = get_legacy_session().get(url)
    df = pd.read_csv(io.BytesIO(res.content), sep=',')
    
    if filter_measure_on_df:
        df = df[df['MEASURE'] == measure]

    df.index = df['TIME']
    
    df_formatted = format_oecd_data(df)
    df_formatted = df_formatted.sort_index()
    check_data_coverage(df_formatted)
    df_formatted.to_csv(f'OECD_{file_name}.csv')
    return df_formatted

In [ ]:
def get_oecd_data(indicator, subject, measure, freq, file_name, 
                  countries_before_subject=False, with_measure=True):
    if freq == 'Q':
        startTime = '1999-Q1'
    else:
        startTime = '1999-01'

    if countries_before_subject:
        countries_and_subject = f'{countries}.{subject}'
    else:
        countries_and_subject = f'{subject}.{countries}'
    
    if with_measure:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{measure}.{freq}/all?startTime={startTime}&contentType=csv'
    else:
        url = f'https://stats.oecd.org/SDMX-JSON/data/{indicator}/{countries_and_subject}.{freq}/all?startTime={startTime}&contentType=csv'

    print(url)
    res = get_legacy_session().get(url)
    df = pd.read_csv(io.BytesIO(res.content), sep=',')
    df.index = df['TIME']
    
    df_formatted = format_oecd_data(df)
    df_formatted = df_formatted.sort_index()
    check_data_coverage(df_formatted)
    df_formatted.to_csv(f'OECD_{file_name}.csv')
    return df_formatted

## Composite Leading Indicators

In [15]:
indicator = 'MEI_CLI' # Composite Leading Indicators (MEI) 
subject = 'BSCICP03' # OECD Standardised Bussiness Confidence Indicator, Amplitude adjusted (Long term average=100), sa
measure = ''
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Bussiness Confidence Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/BSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'South Africa', 'Mexico', 'Indonesia',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
China 2000-02
India 2000-05
Indonesia 2002-03
Chile 2003-11

Missing last:
Russia 2022-01

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,99.23026,96.94991,96.70270,100.00960,99.90664,99.07822,99.07100,99.88649,100.24430,96.24296,...,NaN,109.57260,97.55158,NaN,101.1002,98.82476,94.73983,90.59782,NaN,99.716455
1999-02,99.53894,97.06934,96.99982,100.14230,99.86806,99.27635,98.89312,99.79908,100.25560,96.50874,...,NaN,107.81200,97.48577,NaN,100.9418,98.68932,94.91841,90.41037,NaN,100.154733
1999-03,99.76861,97.21156,97.54137,100.27130,99.78703,99.32484,98.79162,99.93929,100.24350,96.75919,...,NaN,105.40490,97.52265,NaN,100.9921,98.44651,94.99623,90.56648,NaN,100.682359
1999-04,99.95366,97.36842,98.35950,100.43320,99.87154,99.41689,98.76231,100.18510,100.24270,96.92786,...,NaN,103.56070,97.67751,NaN,101.2359,97.84256,95.71694,91.26705,NaN,101.178524
1999-05,100.16460,97.53513,99.12959,100.65620,100.09060,99.49191,98.80769,100.37760,100.25790,97.17252,...,NaN,102.66100,97.72133,NaN,101.2700,97.44176,96.74817,92.18592,NaN,101.719888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,98.89899,100.29390,99.55035,98.02653,99.91919,100.04330,100.58270,99.76989,100.68350,99.38604,...,99.04103,99.93022,98.49326,95.50326,101.8967,98.97838,98.88395,99.50489,NaN,NaN
2023-05,98.88395,100.34110,99.48969,NaN,99.81200,99.88603,100.21450,99.81442,100.52480,100.02480,...,99.05051,100.08220,98.31992,95.46212,101.7273,99.04419,98.06341,99.06139,NaN,NaN
2023-06,98.86854,100.40470,99.79144,NaN,99.73529,99.77724,99.78830,NaN,100.34810,100.39650,...,99.06405,99.63695,98.26650,95.89169,101.7869,98.88184,97.37190,98.75256,NaN,NaN
2023-07,98.89616,NaN,100.19040,NaN,99.51942,99.67213,99.34898,NaN,100.15180,100.48440,...,NaN,99.08010,98.30476,96.50355,101.8207,98.90376,97.30465,NaN,NaN,NaN


In [16]:
subject = 'CSCICP03' # OECD Standardised Consumer Confidence Indicator, Amplitude adjusted (Long term average=100), sa
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Consumer Confidence Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/CSCICP03.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'France', 'Switzerland',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Israel', 'Ireland', 'New Zealand',
       'Austria', 'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Canada', 'Norway', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Israel 2011-03
India 2012-08
Mexico 2001-04
Indonesia 2001-04
Turkiye 2004-01
Poland 2001-05
Chile 2002-03
Colombia 2001-11

Missing last:
India 2020-09

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,102.16050,99.13457,101.76980,NaN,101.25410,101.18270,100.60430,101.25230,100.96870,99.30415,...,NaN,NaN,NaN,NaN,102.41200,100.58040,96.81049,NaN,NaN,90.73000
1999-02,102.37160,99.22217,102.11630,NaN,101.21350,101.01410,100.49530,101.55030,100.89030,99.39217,...,NaN,NaN,NaN,NaN,102.44690,100.56310,96.45536,NaN,NaN,91.17000
1999-03,102.40730,99.25058,102.41340,NaN,101.09430,100.89680,100.28060,101.57250,100.86780,99.56536,...,NaN,NaN,NaN,NaN,102.52370,100.35150,96.24995,NaN,NaN,91.50000
1999-04,102.40250,99.27444,102.53160,NaN,101.00480,100.83500,100.05560,101.54850,100.96440,99.71352,...,NaN,NaN,NaN,NaN,102.75030,100.00990,96.30946,NaN,NaN,91.72000
1999-05,102.44670,99.32959,102.53680,NaN,101.04590,100.85280,99.90978,101.45660,101.09020,99.87493,...,NaN,NaN,NaN,NaN,102.91550,99.80379,96.39417,NaN,NaN,91.87000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,97.18363,98.13555,96.28214,NaN,97.32153,96.76675,98.86586,97.84637,98.43124,98.21471,...,101.6381,99.50516,99.23917,95.58919,98.90280,98.66436,98.23370,95.16866,NaN,99.31689
2023-05,97.17767,98.38691,97.13757,NaN,97.77294,96.81134,98.99891,97.76232,98.33045,98.59695,...,101.7131,99.88246,100.03850,95.79842,99.50943,98.66811,98.28234,95.77176,NaN,99.61455
2023-06,97.48916,98.52669,97.57776,NaN,98.19633,96.87649,98.98434,97.71120,98.29894,98.85841,...,101.6690,98.92008,100.52920,96.05412,99.99138,98.77574,98.44428,96.42462,NaN,NaN
2023-07,97.95027,98.59281,97.54587,NaN,98.43741,96.96812,98.88734,97.74561,98.36086,98.94753,...,101.5384,96.91089,100.85750,96.21217,100.13840,98.92316,98.81435,96.80355,NaN,NaN


In [17]:
subject = 'LOLITOAA' # Amplitude adjusted (CLI)
df = get_oecd_data(indicator, subject, measure, freq, with_measure=False, 
                   file_name='Composite Leading Indicator')
df

https://stats.oecd.org/SDMX-JSON/data/MEI_CLI/LOLITOAA.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Germany', 'Australia', 'Spain', 'Italy', 'China', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye'],
      dtype='object')

No data:
Index(['Switzerland', 'Netherlands', 'Denmark', 'Sweden', 'Finland', 'Belgium',
       'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal',
       'Eurozone', 'Saudi Arabia', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,100.22600,98.08029,99.92517,99.04568,99.87450,NaN,99.32185,100.33960,NaN,NaN,...,93.44679,93.92188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-02,100.43870,98.48177,100.41620,99.32136,99.83424,NaN,99.29385,100.44010,NaN,NaN,...,94.28552,94.92348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03,100.62990,98.87955,100.91410,99.55447,99.86840,NaN,99.32334,100.52450,NaN,NaN,...,95.19625,95.96504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-04,100.79730,99.25786,101.34050,99.73008,100.00550,NaN,99.41097,100.60160,NaN,NaN,...,96.20809,96.85421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-05,100.94080,99.59679,101.66880,99.85417,100.23400,NaN,99.56577,100.66890,NaN,NaN,...,97.27637,97.49230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,98.89024,100.08740,99.22129,97.52503,98.56210,NaN,99.58736,98.78822,NaN,NaN,...,97.78961,100.28580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05,98.97022,100.13520,99.50819,97.51325,98.53325,NaN,99.52377,98.71284,NaN,NaN,...,97.83102,100.30800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06,99.08180,100.17440,99.72255,97.52862,98.51160,NaN,99.41216,98.64871,NaN,NaN,...,97.91052,100.23180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07,99.20307,100.20680,99.88969,97.56876,98.49852,NaN,99.29068,98.60187,NaN,NaN,...,97.98292,100.09870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Key Short-Term Economic Indicators

In [18]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IRSTCI01' # Overnight Interbank Rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Overnight Interbank Rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRSTCI01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'South Africa', 'Mexico', 'Indonesia',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Saudi Arabia', 'Argentina'], dtype='object')

Missing first:

Missing last:
Sweden 2020-10
Russia 2022-08

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.63,0.22526,5.9922,4.991400,3.137000,0.84375,3.137000,4.75,3.137000,3.7699,...,36.43,78.88,15.50,12.94,11.425105,17.9000,9.09,26.98,NaN,60.0
1999-02,4.76,0.17632,5.7693,4.995700,3.118000,1.08036,3.118000,4.75,3.118000,3.5736,...,37.50,77.38,15.50,6.25,10.180684,14.8000,8.00,23.05,NaN,60.0
1999-03,4.81,0.04091,5.5665,4.994300,2.925200,1.00714,2.925200,4.75,2.925200,3.4139,...,37.84,76.32,15.50,6.84,10.147409,15.6000,7.70,22.14,NaN,60.0
1999-04,4.74,0.03143,5.3295,4.777100,2.709100,0.80357,2.709100,4.75,2.709100,3.0343,...,35.19,76.96,15.50,12.81,10.151450,15.3000,7.27,20.72,NaN,60.0
1999-05,4.74,0.03000,5.2689,4.592900,2.551000,0.84052,2.551000,4.75,2.551000,2.8990,...,28.73,76.96,15.50,11.77,10.402900,14.6000,6.91,18.92,NaN,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,4.83,-0.01500,4.1774,4.491718,2.899081,0.90000,2.899081,3.60,2.899081,2.7100,...,5.63,NaN,6.59,11.25,2.899081,17.8314,7.00,13.00,NaN,NaN
2023-05,5.06,-0.05100,4.3528,4.492648,3.046309,0.95000,3.046309,3.83,3.046309,2.9000,...,5.56,NaN,6.74,11.25,3.046309,17.3508,7.00,13.28,NaN,NaN
2023-06,5.08,-0.06600,4.5872,4.683945,3.204590,1.20000,3.204590,4.05,3.204590,3.0900,...,5.61,NaN,6.71,11.25,3.204590,16.0434,7.00,13.28,NaN,NaN
2023-07,5.12,-0.04600,4.9302,4.903890,3.403263,1.20000,3.403263,4.10,3.403263,3.2300,...,5.61,NaN,6.79,11.25,3.403263,15.4338,7.00,13.26,NaN,NaN


In [19]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IR3TIB01' # Short term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Short term interest rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IR3TIB01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Brazil', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
Japan 2002-04
India 2011-12

Missing last:
Turkiye 2008-04
Russia 2022-02

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.89,NaN,5.92171,5.004500,3.1321,NaN,3.1321,4.79,3.1321,3.887500,...,45.50,NaN,12.60,12.75,11.690000,15.8000,8.49,33.23,NaN,43.9
1999-02,4.90,NaN,5.53197,5.033500,3.0934,NaN,3.0934,4.77,3.0934,3.615000,...,38.20,NaN,12.68,10.04,10.664211,15.3000,8.18,30.25,NaN,23.3
1999-03,4.91,NaN,5.42287,5.057391,3.0467,NaN,3.0467,4.81,3.0467,3.459022,...,34.85,NaN,12.84,11.09,10.050455,15.3000,7.58,25.32,NaN,21.6
1999-04,4.88,NaN,5.31773,4.716667,2.6965,NaN,2.6965,4.76,2.6965,3.159773,...,34.09,NaN,12.95,13.59,10.084500,15.4000,7.09,22.08,NaN,20.6
1999-05,4.92,NaN,5.34875,4.615500,2.5790,NaN,2.5790,4.86,2.5790,3.108452,...,31.20,NaN,13.04,11.71,9.945000,14.9000,6.96,19.01,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,5.03,-0.001,4.52000,4.799086,3.1670,1.45,3.1670,3.66,3.1670,3.281571,...,6.76,NaN,6.90,NaN,3.167000,17.1161,7.18,12.53,NaN,NaN
2023-05,5.15,-0.019,4.73000,4.788520,3.3664,1.45,3.3664,3.90,3.3664,3.396137,...,6.76,NaN,6.90,NaN,3.366400,14.8287,7.18,12.57,NaN,NaN
2023-06,5.22,-0.003,5.09000,5.021803,3.5359,1.45,3.5359,4.25,3.5359,3.562376,...,6.75,NaN,6.90,NaN,3.535900,15.9426,7.15,13.02,NaN,NaN
2023-07,5.35,-0.003,5.49000,5.219326,3.6718,1.72,3.6718,4.30,3.6718,3.720800,...,6.75,NaN,6.76,NaN,3.671800,10.3116,7.11,13.58,NaN,NaN


In [20]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'IRLTLT01' # Long-term interest rate
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Long term interest rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/IRLTLT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'South Africa', 'Mexico', 'Indonesia',
       'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia',
       'Russia'],
      dtype='object')

No data:
Index(['Saudi Arabia', 'Turkiye', 'Argentina'], dtype='object')

Missing first:
China 2014-01
India 2011-12
Korea 2000-10
Mexico 2001-07
Indonesia 2012-01
Poland 2001-01
Chile 2004-07
Czechia 2000-04
Colombia 2003-01

Missing last:
Russi

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.72,1.910,4.2868,4.946500,3.7700,2.555,3.702000,5.127895,3.801,4.035,...,NaN,NaN,NaN,NaN,6.30,NaN,NaN,NaN,NaN,96.34
1999-02,5.00,2.117,4.4319,5.124000,3.9290,2.550,3.849000,5.300000,3.932,4.188,...,NaN,NaN,NaN,NaN,6.00,9.27,NaN,NaN,NaN,99.47
1999-03,5.23,1.816,4.6554,5.220435,4.1335,2.604,4.037826,5.526522,4.139,4.432,...,NaN,NaN,NaN,NaN,6.00,9.72,NaN,NaN,NaN,98.67
1999-04,5.18,1.563,4.5920,5.029524,3.9762,2.534,3.850000,5.357500,3.995,4.217,...,NaN,NaN,NaN,NaN,5.90,9.73,NaN,NaN,NaN,110.55
1999-05,5.54,1.334,4.9068,5.359500,4.1600,2.720,4.012105,5.875714,4.193,4.461,...,NaN,NaN,NaN,NaN,5.80,9.72,NaN,NaN,NaN,96.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.46,0.385,3.6503,2.909474,2.9200,1.080,2.359444,3.340000,2.719,2.597,...,6.45,NaN,6.07,5.539444,4.23,8.35,4.65,11.89,NaN,NaN
2023-05,3.57,0.430,3.9621,3.045455,2.9400,0.850,2.342273,3.500000,2.728,2.539,...,6.36,NaN,5.93,5.548636,3.98,7.78,4.50,11.11,NaN,NaN
2023-06,3.75,0.395,4.3659,3.338182,2.9300,0.970,2.376364,3.920000,2.750,2.571,...,6.22,NaN,5.91,5.305500,3.70,7.20,4.35,10.15,NaN,NaN
2023-07,3.90,0.595,4.4372,3.452000,3.0400,0.980,2.456667,4.030000,2.840,2.695,...,6.33,NaN,5.56,5.330500,3.85,7.10,4.15,10.22,NaN,NaN


In [21]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'LRHUTTTT' # Monthly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Monthly Unemployment')
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Korea', 'Mexico', 'Turkiye', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia'],
      dtype='object')

No data:
Index(['Switzerland', 'New Zealand', 'Eurozone', 'China', 'India', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Israel 2012-01
Turkiye 2005-01
Colombia 2007-01

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4.3,4.5,6.2,7.9,12.0,NaN,8.9,7.118793,4.6,5.0,...,NaN,NaN,11.5,8.373607,11.4,7.2,7.8,NaN,NaN,NaN
1999-02,4.4,4.6,6.2,7.9,12.0,NaN,8.8,7.115217,4.5,5.0,...,NaN,NaN,12.0,8.810076,11.4,7.2,8.1,NaN,NaN,NaN
1999-03,4.2,4.7,6.2,7.9,11.8,NaN,8.8,7.012740,4.4,5.0,...,NaN,NaN,12.3,9.311584,11.4,7.1,8.6,NaN,NaN,NaN
1999-04,4.3,4.7,6.1,8.2,11.7,NaN,8.7,6.979468,4.4,5.0,...,NaN,NaN,12.7,10.038610,12.1,7.0,8.6,NaN,NaN,NaN
1999-05,4.2,4.7,6.0,7.9,11.5,NaN,8.6,7.025072,4.3,5.0,...,NaN,NaN,12.9,10.649560,12.0,6.9,8.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,3.4,2.6,4.0,5.0,7.3,NaN,2.9,3.689922,3.4,5.0,...,NaN,10.0,2.8,8.282938,11.6,3.9,2.7,10.288648,NaN,NaN
2023-05,3.7,2.6,4.2,5.2,7.3,NaN,2.9,3.566714,3.5,4.9,...,NaN,9.5,2.8,8.247289,11.2,3.9,2.5,10.096366,NaN,NaN
2023-06,3.6,2.5,4.3,5.4,7.3,NaN,2.9,3.474120,3.5,4.9,...,NaN,9.6,2.8,8.364307,11.0,3.9,2.7,9.365741,NaN,NaN
2023-07,3.5,2.7,NaN,5.5,7.4,NaN,2.9,3.713433,3.6,5.6,...,NaN,9.4,2.8,NaN,10.8,4.0,2.7,9.321362,NaN,NaN


In [22]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'LRHUTTTT' # Quaterly Unemployment Rate - all persons, s.a.
measure = 'ST' # Level, ratio or index
freq = 'Q' # Quaterly
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Quaterly Unemployment')
df.sort_index()

https://stats.oecd.org/SDMX-JSON/data/KEI/LRHUTTTT.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Korea', 'Mexico',
       'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary', 'Czechia',
       'Colombia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'India', 'Brazil', 'Saudi Arabia', 'South Africa',
       'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Turkiye 2005-Q1
Colombia 2007-Q1

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,4.300000,4.600000,6.2,7.900000,11.933333,NaN,8.833333,7.082250,4.500000,5.000000,...,NaN,NaN,11.933333,8.810076,11.400000,7.166667,8.166667,NaN,NaN,NaN
1999-Q2,4.266667,4.733333,6.0,7.900000,11.533333,3.099199,8.633333,6.886058,4.333333,4.966667,...,NaN,NaN,12.933333,10.649560,12.033333,6.933333,8.600000,NaN,NaN,NaN
1999-Q3,4.233333,4.700000,5.9,7.500000,11.133333,NaN,8.466667,6.874120,4.166667,5.166667,...,NaN,NaN,13.833333,10.967844,12.033333,6.733333,8.933333,NaN,NaN,NaN
1999-Q4,4.066667,4.633333,5.8,6.966667,10.666667,NaN,8.300000,6.633702,4.033333,5.300000,...,NaN,NaN,14.800000,10.153666,12.333333,6.666667,9.066667,NaN,NaN,NaN
2000-Q1,4.033333,4.833333,5.8,6.866667,10.233333,NaN,8.133333,6.650521,3.866667,4.666667,...,NaN,NaN,15.800000,9.008904,11.566667,6.566667,9.266667,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q3,3.566667,2.566667,3.6,5.133333,7.200000,4.176039,3.100000,3.501735,3.733333,4.533333,...,NaN,10.033333,2.933333,7.827278,12.300000,3.633333,2.366667,10.946416,NaN,NaN
2022-Q4,3.600000,2.533333,3.7,5.100000,7.200000,4.253417,3.066667,3.472797,3.600000,4.666667,...,NaN,10.133333,2.900000,8.411125,11.666667,3.766667,2.400000,11.041762,NaN,NaN
2023-Q1,3.500000,2.600000,3.9,5.000000,7.100000,4.134205,2.966667,3.581322,3.533333,4.800000,...,NaN,9.933333,2.700000,8.721883,11.266667,3.800000,2.533333,10.625934,NaN,NaN
2023-Q2,3.566667,2.566667,4.2,5.200000,7.300000,3.960959,2.900000,3.576919,3.466667,4.933333,...,NaN,9.700000,2.800000,8.247289,11.266667,3.900000,2.633333,9.916919,NaN,NaN


In [23]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'CPALTT01' # Consumer Prices - Annual inflation
measure = 'GY' # Growth on the same period of the previous year
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   'Inflation Rate')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/CPALTT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone'], dtype='object')

Missing first:
Argentina 2017-12

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,1.670792,0.2,2.0,0.660066,0.244279,0.072915,0.238949,NaN,2.177910,1.703992,...,70.642504,65.901675,6.9,3.594859,3.696482,9.753136,3.587444,17.179262,NaN,96.918604
1999-02,1.605930,-0.1,1.9,0.659341,0.179326,0.288012,0.119190,NaN,2.109408,2.035870,...,53.407429,63.927083,5.6,3.804064,3.689293,9.394904,2.971768,15.379706,NaN,103.242485
1999-03,1.726264,-0.4,2.1,0.987925,0.345269,0.467020,0.238663,NaN,2.147838,2.322206,...,45.416667,63.537401,6.2,4.052286,3.399638,9.274907,2.666667,13.495025,NaN,107.584410
1999-04,2.276923,-0.1,2.0,1.648352,0.421133,0.593769,0.595948,NaN,2.114636,2.364865,...,37.967631,63.854962,6.3,4.033417,2.805966,9.393232,2.511078,11.160355,NaN,113.064572
1999-05,2.088452,-0.4,1.7,1.533406,0.408267,0.575758,0.476190,NaN,2.242028,2.065322,...,30.736258,62.966719,6.4,3.966940,2.372590,8.863680,2.511078,9.998016,NaN,116.711050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,4.930320,NaN,7.8,4.405874,5.884473,2.565343,7.169118,NaN,5.174860,5.282005,...,4.329389,43.684170,14.7,9.907431,3.039219,24.000000,12.710567,12.822871,108.834060,NaN
2023-05,4.047609,NaN,7.9,3.357472,5.119943,2.158705,6.102004,NaN,6.056279,2.928128,...,4.004224,39.585301,13.0,8.726823,2.781513,21.500000,11.060948,12.360256,114.246289,NaN
2023-06,2.969178,NaN,7.3,2.812296,4.531319,1.710877,6.375228,NaN,5.695086,2.464789,...,3.520988,38.213519,11.5,7.560418,1.760478,20.100000,9.703704,12.129618,115.580273,NaN
2023-07,3.177780,NaN,6.4,3.265839,4.288119,1.605105,6.165005,NaN,4.573497,3.133159,...,3.078237,47.831733,10.8,6.475515,2.470822,17.600000,8.771930,11.784920,113.449994,NaN


In [46]:
indicator = 'KEI' # Key Short-Term Economic Indicators
subject = 'CPALTT01' # Consumer Prices - Annual inflation
measure = 'GP' # Growth previous period
freq = 'M' # Monthly
df = get_oecd_data(indicator, subject, measure, freq, 
                   'Inflation Rate MoM')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/CPALTT01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GP.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone'], dtype='object')

Missing first:
Argentina 2017-01

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,0.244051,-0.5,-0.4,0.219058,-0.281366,0.204959,-0.119048,NaN,0.055394,-0.095648,...,2.962227,4.802803,1.5,-0.332803,-1.009651,2.513703,0.726744,2.223074,NaN,8.38
1999-02,0.121729,-0.4,0.2,0.109290,0.307811,0.337974,0.119190,NaN,0.498270,0.765917,...,1.274377,3.173146,0.6,0.071964,-0.482737,1.327434,0.000000,1.687289,NaN,4.13
1999-03,0.303951,0.1,0.5,0.436681,0.332438,0.104550,0.000000,NaN,0.867649,0.475059,...,-0.190658,4.054140,1.0,0.635706,2.214968,1.182678,0.000000,0.940265,NaN,2.79
1999-04,0.727273,0.5,0.5,0.543478,0.280362,0.166821,0.476190,NaN,0.218460,0.283688,...,-0.668577,4.867176,0.8,0.375870,1.040252,1.150872,0.144300,0.785388,NaN,3.03
1999-05,0.000000,0.0,0.2,0.216216,0.012708,-0.224349,0.000000,NaN,0.027248,0.188590,...,-0.288462,2.940834,0.7,0.118176,-0.052034,0.657778,0.000000,0.489308,NaN,2.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,0.505904,NaN,1.2,0.708307,0.607929,-0.003397,0.430663,NaN,1.034317,0.255754,...,0.333595,2.385509,0.7,0.312072,0.637781,0.700000,-0.203390,0.781088,8.402690,NaN
2023-05,0.251844,NaN,0.6,0.383632,-0.051064,0.250977,-0.085763,NaN,0.198397,-1.360544,...,0.087154,0.043076,0.0,0.113817,0.403071,-0.400000,0.271739,0.431244,7.772753,NaN
2023-06,0.322891,NaN,0.2,0.127389,0.178815,0.063622,0.257511,NaN,-0.340567,0.344828,...,0.139324,3.920498,0.0,-0.151584,0.618575,0.300000,0.338753,0.300871,5.950832,NaN
2023-07,0.190752,NaN,-0.3,0.572519,0.050999,-0.141837,0.256849,NaN,1.033140,1.804124,...,0.208696,9.488824,-0.2,0.349173,-1.145123,0.300000,0.472654,0.501727,6.344851,NaN


In [24]:
subject = 'B6BLTT02' # Current Account as a % of GDP
measure = 'ST' # Level, ratio or index
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Current Account')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/B6BLTT02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ST.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Netherlands 2003-Q2
Denmark 2005-Q1
Belgium 2003-Q1
Ireland 2002-Q1
Saudi Arabia 2006-Q1
Mexico 2006-Q1
Indonesia 2004-Q1
Poland 2004-Q1
Chile 2003-Q1
Greece

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,-2.585638,2.328677,-2.767561,-0.146836,3.757485,2.048831,-1.989932,-5.629524,NaN,NaN,...,NaN,3.256730,NaN,NaN,NaN,-7.830177,-2.908802,NaN,NaN,NaN
1999-Q2,-2.853475,2.460816,-2.203467,-0.252270,3.549500,10.159241,-0.791376,-5.935598,NaN,NaN,...,NaN,-1.226594,NaN,NaN,NaN,-9.332882,-1.883920,NaN,NaN,NaN
1999-Q3,-3.143675,2.575973,-2.207767,0.724169,2.261773,11.742319,-1.359191,-5.656639,NaN,NaN,...,NaN,-0.891263,NaN,NaN,NaN,-8.376171,-0.589354,NaN,NaN,NaN
1999-Q4,-3.300348,2.523838,-1.304133,0.360043,3.720123,10.067650,-1.736559,-4.705191,NaN,NaN,...,NaN,-1.336144,NaN,NaN,NaN,-8.137142,-3.115128,NaN,NaN,NaN
2000-Q1,-3.790054,2.619592,-1.587838,2.086362,1.056238,8.593989,-1.189250,-4.840895,NaN,NaN,...,NaN,-2.774978,NaN,NaN,NaN,-8.108712,-3.747868,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,-3.941371,2.290976,-4.562042,0.684377,-1.282522,6.182092,4.000580,1.973024,6.354128,13.162071,...,0.695421,-3.308629,-3.483302,-12.445438,-10.431840,-7.407750,-6.314773,-5.934337,-1.345756,NaN
2022-Q3,-3.463808,0.744857,-2.032066,-1.399075,-3.192699,14.023609,3.929420,-0.122180,10.298289,16.204049,...,0.575878,-6.925721,-1.994737,-8.704062,-6.976852,-10.365228,-7.221750,-7.161150,-2.059779,NaN
2022-Q4,-3.307890,1.759062,-0.388202,-1.156630,-2.391751,7.806503,4.238211,1.728127,7.614000,12.866754,...,2.195254,-4.887727,-2.154689,-6.013019,-13.418218,-8.013661,-5.790306,-6.168764,0.970018,NaN
2023-Q1,-3.306474,1.915240,-1.662314,-0.452213,-1.160198,9.852435,5.347567,1.941856,10.856231,12.854511,...,1.080092,-6.457548,2.371582,0.610972,1.098207,-3.683799,-0.809308,-3.561610,-2.892637,NaN


In [25]:
subject = 'PRINTO01' # Industrial production
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'M'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Industrial Production')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/PRINTO01.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'India', 'Korea', 'Brazil', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'Saudi Arabia', 'South Africa',
       'Mexico', 'Indonesia', 'Argentina'],
      dtype='object')

Missing first:
Switzerland 2011-10

Missing last:
Russia 2021-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,3.638185,-6.206206,2.144470,5.455559,1.415701,NaN,0.400534,NaN,1.085776,1.806589,...,NaN,-5.990783,-4.123498,-5.357759,5.307198,13.445378,-8.702718,-14.437933,NaN,-2.243915
1999-02,4.058844,-3.699897,0.783875,4.073814,0.412939,NaN,-1.204819,NaN,3.193833,0.858369,...,NaN,-5.517241,-5.170261,-4.866265,2.622234,5.548038,-7.656236,-17.851952,NaN,-9.360208
1999-03,4.243102,-0.315126,1.339286,2.673290,1.542135,NaN,-1.849406,NaN,3.837719,1.052632,...,NaN,-4.977376,1.832251,-3.455285,-5.578570,8.502024,-9.609728,-22.563792,NaN,-5.861282
1999-04,4.122832,-1.789474,1.230425,3.755275,1.031130,NaN,-0.266667,NaN,0.867679,0.333704,...,NaN,-4.387991,-0.036993,-5.374827,-2.161938,8.010681,-7.047759,-19.893567,NaN,-5.771179
1999-05,4.075301,0.530223,3.636364,4.395719,0.174588,NaN,-0.132626,NaN,-0.214823,3.543743,...,NaN,-3.386005,3.220337,-3.431228,3.566775,4.427083,-1.845239,-20.107370,NaN,-1.137348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,0.323069,0.190476,-1.313321,0.719543,1.514995,NaN,2.531646,NaN,-11.695376,10.162917,...,NaN,-0.889802,-3.405370,-1.012847,4.506066,-5.693665,1.687389,-4.755581,NaN,NaN
2023-05,0.009243,2.300000,-2.061856,0.605288,2.238193,NaN,1.668405,NaN,-8.820799,6.495468,...,NaN,0.410397,-2.316347,-5.392686,1.684920,-4.313099,0.612423,-3.345041,NaN,NaN
2023-06,-0.310655,-0.189573,0.757576,-0.393338,-0.262892,NaN,0.000000,NaN,-6.308851,11.483967,...,NaN,0.202293,-0.935204,-2.218562,-3.640777,-5.859066,0.344234,-4.805130,NaN,NaN
2023-07,-0.039561,-2.358491,0.475285,NaN,2.674515,NaN,-1.352758,NaN,-6.454630,4.217791,...,NaN,6.931408,-2.200000,-2.002862,-2.168675,-2.627389,-2.321582,NaN,NaN,NaN


In [26]:
subject = 'NAEXKP02' # Private Consumption
measure = 'GY' # Growth on the same period of the prevoius year
freq = 'Q'
df = get_oecd_data(indicator, subject, measure, freq, 
                   file_name='Private Consumption')
df

https://stats.oecd.org/SDMX-JSON/data/KEI/NAEXKP02.AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.GY.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Saudi Arabia', 'Argentina'], dtype='object')

Missing first:
India 2012-Q2
Colombia 2006-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,5.651322,0.861539,3.456293,3.576515,3.023039,2.191664,2.650972,6.490941,6.640931,1.752536,...,2.188967,-0.725987,4.109588,-3.043256,3.051948,6.109058,3.365542,NaN,NaN,NaN
1999-Q2,5.413038,1.039537,4.753388,3.362945,2.821045,2.099397,3.266317,5.495034,6.322037,-0.222539,...,3.892209,3.527555,6.221875,-4.940129,2.276205,6.912400,3.523816,NaN,NaN,NaN
1999-Q3,5.225906,1.043997,4.130955,4.047948,4.000820,2.025536,2.008873,4.613198,5.689979,-0.513521,...,7.656950,-1.874710,4.484617,-0.730095,3.114917,7.079424,3.645058,NaN,NaN,NaN
1999-Q4,5.244597,1.470688,4.878311,4.743512,3.845444,2.284799,1.993951,5.055635,5.013944,-1.447840,...,4.899022,-0.539694,6.698273,4.840225,1.424292,6.508055,1.928596,NaN,NaN,NaN
2000-Q1,5.749823,1.647361,5.156592,4.343575,4.771746,2.159169,1.646271,3.477669,4.753546,-0.019015,...,0.806698,3.610387,4.363981,5.015551,2.407057,4.243081,1.268848,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,2.365147,2.559188,4.826089,7.926034,4.760390,4.331350,5.264206,5.295827,7.001637,-1.462915,...,4.880922,21.010991,6.386004,7.478210,8.829624,8.034160,-0.130161,14.910579,NaN,NaN
2022-Q3,2.192283,3.746768,1.633502,3.138237,-0.030272,2.652657,1.605397,11.547891,2.890485,-3.252893,...,5.212993,18.131735,2.024316,-3.414986,5.481078,4.328660,-4.543280,9.129451,NaN,NaN
2022-Q4,1.668982,0.955686,1.648990,3.039665,-0.883391,2.996229,0.512529,5.198754,4.237358,-4.918704,...,5.059106,15.863127,-1.412260,-4.409712,3.957130,1.234183,-5.258664,3.922484,NaN,NaN
2023-Q1,2.366445,2.617380,0.257922,3.756974,0.410875,3.119141,-0.368062,3.562526,3.174298,-1.067334,...,5.027742,16.847549,-2.562805,-7.135776,2.496861,-2.744051,-5.322971,3.055144,NaN,NaN


## Quaterly National Accounts

In [29]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # Gross domestic product - expenditure approach
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Quaterly GDP YoY')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2011-Q1
Colombia 2006-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,4.821858,-0.840503,2.193433,4.194104,2.784202,1.030432,0.793797,5.173796,4.213417,2.311058,...,-6.407874,-4.090021,2.100109,-2.237507,2.361704,2.520821,0.395327,NaN,-2.176492,NaN
1999-Q2,4.727532,-0.033774,2.063279,5.009661,2.617900,0.571309,1.220621,4.541221,4.624423,3.621945,...,1.497007,-3.864924,4.224077,-3.605963,2.861976,2.067932,0.575203,NaN,-5.140582,NaN
1999-Q3,4.800205,0.330989,3.467173,5.690714,3.529290,1.175652,1.859483,3.776954,5.167647,2.381317,...,2.921313,-4.137498,5.298836,-0.906906,2.896443,3.041622,1.473288,NaN,-5.124917,NaN
1999-Q4,4.827627,-0.435648,4.314616,5.738707,4.179188,3.735687,3.124281,3.962759,5.680132,3.481499,...,5.991807,-1.017627,6.529382,5.105253,3.724339,4.188502,2.542947,NaN,-1.014421,NaN
2000-Q1,4.225956,2.667340,4.774235,5.516373,4.357079,4.320257,3.399247,3.623640,4.594874,3.552325,...,3.483446,2.265206,6.377840,6.077906,3.330422,5.153192,3.879361,NaN,-0.106447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,1.796042,1.434724,3.841392,4.719747,3.875825,3.288302,1.555184,3.092582,5.161248,3.151989,...,5.288928,6.987693,5.977937,5.499798,7.123086,6.229507,3.359600,12.151756,7.688863,NaN
2022-Q3,1.942138,1.520914,1.976076,3.818950,1.180414,1.377187,1.245724,5.994062,2.994482,2.284758,...,6.111392,3.782324,5.006772,0.007175,4.050557,3.744120,1.350071,7.332106,5.342538,NaN
2022-Q4,0.881278,0.461850,0.573879,2.074066,0.719408,1.153463,0.824963,2.696466,3.061577,0.372316,...,5.157855,3.354782,0.990492,-2.137102,4.754101,0.683511,0.120022,1.970428,1.102267,NaN
2023-Q1,1.800193,1.849551,0.231021,2.079295,0.826340,1.443426,-0.268399,2.444471,2.088185,2.029691,...,4.940213,3.005379,-0.618343,-0.957612,1.968942,-1.195203,-0.422889,2.998554,1.225916,NaN


In [30]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # Gross domestic product - expenditure approach
measure = 'GPSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Quaterly GDP QoQ')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.GPSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
China 2011-Q1
Saudi Arabia 2010-Q2
Colombia 2005-Q2
Russia 2003-Q2

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,0.943827,-1.382290,0.717533,1.813816,0.663422,0.103070,1.157633,0.730019,1.666754,0.906036,...,5.647681,-1.685820,0.804492,0.019194,0.762331,-0.019034,0.026331,NaN,-1.245853,NaN
1999-Q2,0.835000,0.374807,0.404130,0.836545,0.626081,0.673753,0.000000,0.324950,1.112902,0.516383,...,-0.054983,-0.563716,2.094611,0.272837,0.933205,1.107577,0.453881,NaN,-1.129907,NaN
1999-Q3,1.323027,0.544253,1.680631,1.564676,1.469640,0.566115,1.107427,1.158502,1.377229,0.618053,...,0.273417,-1.686618,2.000763,2.109523,0.382798,1.892282,0.923652,NaN,-0.288150,NaN
1999-Q4,1.642779,0.038457,1.449585,1.406696,1.359066,2.355895,0.827551,1.696859,1.406875,1.398796,...,0.107212,2.987420,1.480659,2.634063,1.598925,1.152698,1.118770,NaN,1.672821,NaN
2000-Q1,0.364447,1.691195,1.161304,1.599736,0.835310,0.667169,1.427355,0.401445,0.622710,0.975100,...,3.147464,1.574851,0.661094,0.944780,0.379663,0.906698,1.329947,NaN,-0.340002,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,-0.144634,1.291829,0.051298,0.896973,0.379176,0.870865,-0.129573,0.723268,1.841992,0.868664,...,0.912726,1.414399,-1.389859,-0.166120,0.504221,0.798849,0.152875,1.588398,1.712513,NaN
2022-Q3,0.801155,-0.297841,-0.093939,0.558360,0.314971,0.262931,0.361414,0.674356,-0.185737,-0.096688,...,0.820842,0.542207,1.239117,-1.231860,0.264727,-0.954201,-0.225657,0.859365,0.453445,NaN
2022-Q4,0.637398,0.050687,0.129018,-0.037225,0.110836,0.021390,-0.406273,0.652831,0.864362,0.509287,...,2.305180,1.148543,-2.014221,0.038866,1.193791,-0.636148,-0.391728,-1.660601,-1.797086,NaN
2023-Q1,0.496812,0.799893,0.144622,0.647511,0.019235,0.282439,-0.092722,0.372296,-0.432352,0.735889,...,0.820317,-0.126048,1.595215,0.405503,-0.004243,-0.400312,0.041874,2.221733,0.885354,NaN


In [31]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'P6' # Export of goods and services
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Export Growth YoY')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.P6.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina',
       'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Saudi Arabia'], dtype='object')

Missing first:
Colombia 2006-Q1
Argentina 2005-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,1.781989,-3.184403,-0.617973,11.156666,2.646355,-0.971246,-0.427908,3.412682,8.100655,10.067144,...,-43.735575,-6.354879,-6.384093,6.630604,27.603200,7.720736,-4.684830,NaN,NaN,NaN
1999-Q2,4.175486,-0.334058,0.502661,10.093074,2.347465,-0.323806,3.043550,1.203424,8.500125,14.364930,...,-37.454740,-13.314257,-5.721030,7.064712,34.606750,7.845972,5.518124,NaN,NaN,NaN
1999-Q3,7.807311,3.557829,5.643853,11.674287,5.263250,6.463969,6.527555,5.613551,8.203445,6.928990,...,-38.628370,-12.615323,1.427944,6.754554,11.704600,12.210165,8.312234,NaN,NaN,NaN
1999-Q4,6.180181,7.880496,7.765527,10.402491,8.439193,7.602412,10.619864,8.558975,9.982348,14.006413,...,12.318635,-9.235007,0.564571,7.502319,26.449448,19.100028,10.843612,NaN,NaN,NaN
2000-Q1,8.030438,13.420954,10.048094,10.125817,12.360891,12.257600,15.194693,12.100828,11.010783,7.523480,...,14.728537,7.905877,20.557283,8.342681,29.163627,23.714140,16.355738,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,7.325643,2.933894,5.618981,4.569915,7.754029,-2.273031,3.789304,4.019504,5.923527,11.490010,...,18.320344,17.685666,6.504104,0.611515,12.363434,10.173475,3.322099,27.887708,8.413205,NaN
2022-Q3,11.343816,5.872232,21.459001,6.442725,7.269349,10.229457,5.492632,6.272277,2.955846,14.819292,...,18.645208,11.055183,8.482966,3.501379,-2.656981,16.334491,12.722731,16.213671,-2.681259,NaN
2022-Q4,4.618863,7.338059,6.177992,3.076530,4.177111,3.984386,0.698683,8.183728,4.604533,7.894932,...,12.529061,-3.651603,4.326740,0.666062,-2.015604,12.177629,10.853838,-0.524894,8.334779,NaN
2023-Q1,7.858563,1.864150,6.590961,7.894554,0.697370,3.738095,1.263485,10.866473,2.625743,8.329034,...,9.803995,-2.736896,2.956394,2.659233,7.105696,6.332292,6.443421,2.571810,-4.289057,NaN


In [32]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'P7' # Import of goods and services
measure = 'GYSA' # Growth rate based on seasonally adjusted volume data, percentage change on the same quarter the previous year
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Import Growth YoY')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.P7.GYSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'India', 'Korea',
       'Brazil', 'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Czechia', 'Colombia', 'Argentina',
       'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'Saudi Arabia'], dtype='object')

Missing first:
Colombia 2006-Q1
Argentina 2005-Q1
Russia 2004-Q1

Missing last:
Russia 2021-Q3

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,10.051768,-3.659031,7.524780,4.195287,5.163032,-1.693784,7.044053,4.125138,6.834834,2.149968,...,-52.586894,-16.127535,6.248736,-14.299451,11.243866,11.682590,-0.527630,NaN,NaN,NaN
1999-Q2,10.675835,2.658435,5.796324,5.572196,4.677758,2.947450,7.649275,5.846964,9.203847,1.325768,...,-44.795896,-0.232669,3.863744,-14.624429,22.116380,11.794640,5.914826,NaN,NaN,NaN
1999-Q3,13.150121,5.085316,8.194352,10.627752,6.257972,8.002020,9.592060,11.090938,9.863793,1.410297,...,-43.129505,-3.641717,-1.399095,-10.898306,14.415064,12.097296,3.789395,NaN,NaN,NaN
1999-Q4,12.406205,10.883490,9.975970,11.799637,9.400408,7.537810,10.068958,12.367756,12.077740,5.274007,...,-9.014625,7.588306,-3.567265,1.427644,11.484682,17.258219,10.806590,NaN,NaN,NaN
2000-Q1,13.444215,7.603196,7.642644,12.046229,13.373033,11.600728,9.256413,13.029562,11.331291,10.038261,...,7.342299,27.188105,13.582338,9.287916,21.740922,22.518018,13.609156,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,11.227663,3.464954,18.193128,11.323366,8.028372,1.585265,7.762326,9.924576,4.926281,7.643395,...,16.454209,7.515869,8.200372,10.861456,14.601612,9.649723,3.040119,25.525833,26.042557,NaN
2022-Q3,7.393347,10.934706,9.707575,11.534171,11.735701,13.047504,10.208940,19.414848,1.897210,7.332938,...,19.243546,13.656709,7.143819,-2.944073,5.218881,13.663507,8.083188,21.627847,21.455648,NaN
2022-Q4,1.457677,10.446743,5.185368,2.966873,5.558306,3.471510,2.588815,12.758977,4.152577,4.517525,...,5.251138,9.845828,0.965146,-15.763071,6.786512,10.175829,8.252322,7.425617,0.765522,NaN
2023-Q1,-2.261903,4.128203,-6.461183,2.654120,1.057715,9.503997,1.011091,5.675720,2.326033,1.105629,...,4.290397,14.633296,-4.023937,-16.868066,5.163947,1.856180,2.979052,-7.591450,5.220883,NaN


In [36]:
indicator = 'QNA' # Quaterly National Accounts
subject = 'B1_GE' # GDP per capita
measure = 'HVPVOBARSA' # Per Head, US dollars, volume estimates, fixed PPPs, OECD reference year (2015), seasonally adjusted
freq = 'Q' # Quaterly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='GDP per capita')
df

https://stats.oecd.org/SDMX-JSON/data/QNA/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE.HVPVOBARSA.Q/all?startTime=1999-Q1&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Korea',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Poland', 'Chile', 'Greece',
       'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'China', 'India', 'Brazil', 'Indonesia', 'Turkiye',
       'Argentina'],
      dtype='object')

Missing first:
Japan 2007-Q3
Saudi Arabia 2010-Q1
South Africa 2008-Q1
Mexico 2005-Q1
Colombia 2001-Q1
Russia 2003-Q1

Missing last:
Saudi Arabia 2020-Q4
Russia 

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-Q1,46719.9,NaN,35205.0,36810.5,35777.0,54309.0,38826.6,36641.5,42787.7,43465.4,...,NaN,NaN,14511.0,13691.5,27526.6,17633.9,22153.5,NaN,NaN,NaN
1999-Q2,46981.4,NaN,35315.9,37051.6,35950.6,54600.7,38821.3,36678.9,43203.6,43657.0,...,NaN,NaN,14816.1,13674.3,27765.8,17843.0,22260.6,NaN,NaN,NaN
1999-Q3,47452.0,NaN,35878.3,37524.2,36423.2,54835.0,39240.6,36994.9,43715.6,43885.6,...,NaN,NaN,15114.2,13912.6,27853.9,18191.4,22464.0,NaN,NaN,NaN
1999-Q4,48087.0,NaN,36367.4,37938.7,36856.9,56050.4,39562.9,37520.2,44246.8,44466.0,...,NaN,NaN,15339.2,14231.1,28281.0,18415.4,22719.8,NaN,NaN,NaN
2000-Q1,48140.1,NaN,36757.8,38503.0,37098.4,56347.6,40121.7,37536.9,44452.2,44865.9,...,NaN,NaN,15442.2,14318.1,28359.0,18597.5,23037.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-Q2,62487.8,42410.2,44112.7,46352.8,42911.8,70248.5,50069.3,50297.2,56515.7,55689.6,...,NaN,NaN,35412.4,23888.4,29697.7,34585.8,38507.7,15024.7,NaN,NaN
2022-Q3,62898.2,42344.1,44062.8,46270.4,43011.2,70277.5,50119.9,50373.4,56185.1,55447.7,...,NaN,NaN,35873.0,23552.4,29811.4,34264.6,38428.0,15115.0,NaN,NaN
2022-Q4,63207.7,42431.8,44110.6,45826.5,43022.9,70109.0,49842.1,50456.9,56502.0,55645.5,...,NaN,NaN,35178.4,23522.4,30225.4,34054.3,38270.3,14824.3,NaN,NaN
2023-Q1,63453.6,42868.1,44030.3,45803.9,42995.2,69890.3,49655.5,50417.1,56163.0,55960.6,...,NaN,NaN,35777.4,23578.5,30260.1,34205.9,38225.5,15113.5,NaN,NaN


### Gross domestic product - Current prices - Yearly

In [35]:
indicator = 'SNA_TABLE1' # Gross domestic product
subject = 'B1_GE' # GDP Current Prices
measure = 'VXCOB' # Current Prices, constant exchange rate, OECD base year, (US Dollar, Million, 2015)
freq = '' # Yearly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='GDP Current Prices', with_measure=False,
                   filter_measure_on_df=True)
df

https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE./all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index([], dtype='object')

Missing first:
Argentina 2004

Missing last:
United States 2022
Japan 2022
United Kingdom 2022
Canada 2022
France 2022
Switzerland 2022
Germany 2022
Australia 2022
Nether

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999,9631172.0,4362626.714,1595229.107,788190.393,1554426.450,464956.846,2285019.608,497456.873,465395.168,184532.987,...,98741.812,39475.706,178944.157,58798.913,148440.319,41661.983,91589.329,60950.313,NaN,84925.900
2000,10250952.0,4423330.247,1682301.743,864938.171,1640509.117,489972.140,2340062.542,531065.413,501507.593,197225.012,...,113573.507,63049.144,198562.939,64536.731,156715.674,47699.612,97008.547,71161.237,NaN,128635.403
2001,10581929.0,4392235.775,1749798.963,895022.950,1706652.728,502541.876,2410461.135,567981.016,534653.181,203856.304,...,134539.224,90906.409,207334.660,69419.634,168861.025,55126.775,104847.848,77621.060,NaN,157475.640
2002,10929108.0,4332958.170,1820370.071,933458.616,1761716.744,500921.187,2438842.475,603034.988,556017.961,209615.124,...,148882.206,133128.130,215469.956,74036.355,181361.825,62412.569,109395.071,84935.412,NaN,190700.479
2003,11456450.0,4328744.001,1924503.401,981201.546,1809244.947,506666.645,2453765.424,648806.489,568969.305,213551.001,...,164559.678,173592.020,224738.559,80867.438,198497.133,68484.731,114780.304,96507.084,NaN,232809.603
2004,12217196.0,4373622.713,2021891.591,1044530.349,1890631.046,521898.821,2510295.105,694863.471,587249.640,223843.884,...,187617.413,214283.445,247537.074,92324.629,214930.203,75573.229,125177.377,109614.735,52540.393,300122.909
2005,13039197.0,4399354.673,2138343.393,1111671.361,1959294.361,541207.835,2538909.443,750440.468,611211.790,235732.184,...,226717.268,250100.630,262774.904,104671.180,221061.913,80889.141,133567.803,123262.185,63091.787,380895.811
2006,13815583.0,4421285.536,2250167.555,1170331.675,2050547.358,575366.059,2646277.005,817588.432,648561.322,250042.138,...,272884.428,292556.843,283706.327,124712.628,241720.218,87155.662,143539.042,139181.031,77536.000,474445.188
2007,14474228.0,4455252.516,2361626.579,1233717.564,2153963.945,612111.996,2773282.946,885855.027,686977.096,258452.642,...,322871.288,326364.570,315030.641,137832.380,258177.650,92155.041,156899.558,156287.426,97147.422,586023.874
2008,14769862.0,4360593.569,2436403.659,1295792.053,2210571.293,638424.397,2825363.481,947539.799,718074.524,267760.829,...,404412.197,368659.321,341045.497,143500.482,268491.457,97553.790,164352.254,173811.798,124512.400,727549.725


In [37]:
indicator = 'SNA_TABLE1' # Gross domestic product
subject = 'B1_GE' # GDP Current Prices
measure = 'VPCOB' # Current Prices, constant PPPs, OECD base year, (US Dollar, Million, 2015)
freq = '' # Yearly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='GDP Current Prices PPP', with_measure=False,
                   filter_measure_on_df=True)

https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.B1_GE./all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China',
       'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index([], dtype='object')

Missing first:
Argentina 2004

Missing last:
United States 2022
Japan 2022
United Kingdom 2022
Canada 2022
France 2022
Switzerland 2022
Germany 2022
Australia 2022
Nether

## Main Economic Indicators

In [42]:
indicator = 'MEI' # Main Economic Indicators
subject = 'CPALTT01' # Customer Price Index - All Items Total
measure = 'IXOB' # Index 2015 = 100
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Customer Price Index')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.CPALTT01.IXOB.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia', 'Russia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'Eurozone', 'Argentina'], dtype='object')

Missing first:

Missing last:
Japan 2021-06
Russia 2022-03

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,69.319922,99.872741,71.8,72.293916,77.97,91.350764,78.272042,NaN,72.25,72.651079,...,32.359971,7.240028,61.751419,59.105552,68.140908,45.124512,69.3,43.514440,NaN,14.599048
1999-02,69.404304,99.465513,71.9,72.372926,78.21,91.659505,78.365334,NaN,72.61,73.207525,...,32.772359,7.469765,62.103078,59.148087,67.811967,45.723510,69.3,44.248654,NaN,15.201989
1999-03,69.615260,99.567320,72.3,72.688965,78.47,91.755335,78.365334,NaN,73.24,73.555305,...,32.709876,7.772600,62.736065,59.524095,69.313981,46.264272,69.3,44.664709,NaN,15.626125
1999-04,70.121552,100.076355,72.6,73.084014,78.69,91.908403,78.738502,NaN,73.40,73.763972,...,32.491186,8.150906,63.228389,59.747828,70.035021,46.796715,69.4,45.015500,NaN,16.099596
1999-05,70.121552,100.076355,72.8,73.242033,78.70,91.702208,78.738502,NaN,73.42,73.903084,...,32.397461,8.390610,63.650380,59.818436,69.998579,47.104533,69.4,45.235764,NaN,16.457007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,127.992085,NaN,128.3,123.571240,117.50,106.241120,122.929506,NaN,126.01,117.600000,...,127.377410,498.892876,147.008547,143.592072,114.344323,159.141377,147.2,155.217990,NaN,NaN
2023-05,128.314425,NaN,129.1,124.045299,117.44,106.507761,122.824078,NaN,126.26,116.000000,...,127.488424,499.107777,147.008547,143.755505,114.805212,158.549223,147.6,155.887358,NaN,NaN
2023-06,128.728741,NaN,129.4,124.203318,117.65,106.575523,123.140363,NaN,125.83,116.400000,...,127.666046,518.675288,147.008547,143.537594,115.515368,158.993338,148.1,156.356377,NaN,NaN
2023-07,128.974293,NaN,129.0,124.914406,117.71,106.424360,123.456648,NaN,127.13,118.500000,...,127.932480,567.891475,146.682947,144.038788,114.192575,159.437454,148.8,157.140860,NaN,NaN


In [43]:
indicator = 'MEI' # Main Economic Indicators
subject = 'MANMM101' # Narrow Money - M1 and components
measure = 'ST' # National Currency
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Narrow Money M1')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.MANMM101.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'Switzerland',
       'Australia', 'Denmark', 'Sweden', 'Norway', 'Israel', 'New Zealand',
       'Eurozone', 'China', 'India', 'Korea', 'Brazil', 'South Africa',
       'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Hungary',
       'Czechia', 'Russia'],
      dtype='object')

No data:
Index(['France', 'Germany', 'Netherlands', 'Spain', 'Italy', 'Finland',
       'Belgium', 'Ireland', 'Austria', 'Portugal', 'Saudi Arabia', 'Greece',
       'Colombia', 'Argentina'],
      dtype='object')

Missing first:

Missing last:
New Zealand 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,1103.9,2.503207e+06,359951.0,203.111,NaN,201287.0,NaN,115.004,NaN,381467.0,...,101.953,3.511547e+03,88635.8,2920.07000,NaN,1822.583988,475.189017,NaN,NaN,177.9
1999-02,1085.2,2.462410e+06,362493.0,200.414,NaN,206120.0,NaN,114.476,NaN,375890.0,...,103.458,3.516057e+03,91943.2,2945.45000,NaN,1845.467384,481.867884,NaN,NaN,180.6
1999-03,1097.5,2.577053e+06,374161.0,199.059,NaN,203007.0,NaN,113.960,NaN,373044.0,...,105.705,4.545328e+03,95283.3,2903.54000,NaN,1884.600523,482.115249,NaN,NaN,174.0
1999-04,1113.6,2.632567e+06,369023.0,203.614,NaN,209720.0,NaN,115.529,NaN,397591.0,...,100.708,3.944113e+03,93989.0,2882.30000,NaN,1881.242901,488.794116,NaN,NaN,195.1
1999-05,1096.4,2.670634e+06,372564.0,207.212,NaN,210485.0,NaN,115.803,NaN,404926.0,...,103.301,3.923985e+03,97092.0,2932.15000,NaN,1989.088947,499.307148,NaN,NaN,205.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,18746.9,1.069616e+07,2317997.0,1507.014,NaN,698014.0,NaN,1624.961,NaN,1565234.7,...,4673.100,3.676969e+06,1548800.0,55688.39419,NaN,31201.779630,4823.123400,NaN,NaN,NaN
2023-05,18501.6,1.073424e+07,2314475.0,1505.984,NaN,683123.0,NaN,1620.659,NaN,1546405.3,...,4623.100,3.523187e+06,1555670.0,55657.70695,NaN,30966.006910,4913.592400,NaN,NaN,NaN
2023-06,18449.5,1.072017e+07,2304395.0,1509.464,NaN,668230.0,NaN,1621.039,NaN,1530061.1,...,4682.800,4.180491e+06,1554380.0,55363.04546,NaN,30465.009870,4907.500000,NaN,NaN,NaN
2023-07,18355.0,1.071008e+07,NaN,NaN,NaN,651358.0,NaN,1617.867,NaN,1544155.3,...,4648.200,4.390129e+06,1571610.0,54720.30532,NaN,NaN,4947.900000,NaN,NaN,NaN


In [44]:
indicator = 'MEI' # Main Economic Indicators
subject = 'MABMM301' # Broad Money - M3 and components
measure = 'ST' # National Currency
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Broad Money M3')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.MABMM301.ST.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'Switzerland',
       'Australia', 'Denmark', 'Sweden', 'Norway', 'Israel', 'New Zealand',
       'Eurozone', 'China', 'India', 'Korea', 'Brazil', 'South Africa',
       'Mexico', 'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Hungary',
       'Czechia', 'Russia'],
      dtype='object')

No data:
Index(['France', 'Germany', 'Netherlands', 'Spain', 'Italy', 'Finland',
       'Belgium', 'Ireland', 'Austria', 'Portugal', 'Saudi Arabia', 'Greece',
       'Colombia', 'Argentina'],
      dtype='object')

Missing first:

Missing last:
Czechia 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,4409.5,9.415504e+06,817074.0,597.640,NaN,477485.0,NaN,386.059,NaN,538381.0,...,596.541,2.320021e+04,226342.7,31230.5800,NaN,4523.179866,1229.7,NaN,NaN,644.0
1999-02,4418.7,9.364157e+06,824741.0,595.777,NaN,483479.0,NaN,387.433,NaN,525330.0,...,602.666,2.420893e+04,233423.8,31216.2700,NaN,4592.524586,1247.4,NaN,NaN,665.8
1999-03,4455.6,9.431088e+06,832631.0,598.793,NaN,481796.0,NaN,388.709,NaN,518872.0,...,603.325,2.597040e+04,236748.5,31144.8200,NaN,4581.527587,1247.9,NaN,NaN,683.7
1999-04,4501.4,9.481738e+06,835921.0,606.502,NaN,484440.0,NaN,392.783,NaN,550505.0,...,613.140,2.735291e+04,236784.4,31502.1300,NaN,4570.787240,1261.5,NaN,NaN,726.8
1999-05,4464.1,9.506468e+06,844372.0,608.041,NaN,484532.0,NaN,397.895,NaN,547787.0,...,628.260,2.909660e+04,240024.5,32249.6600,NaN,4663.614757,1271.9,NaN,NaN,765.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,20865.8,1.587349e+07,3543703.0,3459.862,NaN,1153875.0,NaN,2868.510,NaN,1889329.4,...,8352.300,9.510649e+06,2147520.0,318732.0483,NaN,40982.490560,NaN,NaN,NaN,NaN
2023-05,20749.0,1.590454e+07,3558870.0,3461.115,NaN,1142752.0,NaN,2874.758,NaN,1878493.6,...,8332.300,9.409262e+06,2157900.0,322092.0552,NaN,40797.698570,NaN,NaN,NaN,NaN
2023-06,20802.3,1.594298e+07,3563073.0,3470.763,NaN,1133624.0,NaN,2871.678,NaN,1864208.0,...,8373.000,1.083449e+07,2157330.0,322497.1209,NaN,39986.671550,NaN,NaN,NaN,NaN
2023-07,20783.6,1.594625e+07,NaN,NaN,NaN,1135428.0,NaN,2896.591,NaN,1880333.5,...,8350.500,1.177296e+07,2173200.0,322404.9335,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
indicator = 'MEI' # Main Economic Indicators
subject = 'XTEXVA01' # Export - Value (goods)
measure = 'CXML' # US Dollars, monthly level
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Export Value')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.XTEXVA01.CXML.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2016-05

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,52.436,30.493097,19.288241,17.498198,24.622241,5.746829,41.624862,4.000267,17.478985,3.878038,...,3.0178,1.883315,2.008984,1.367989,0.890625,1.667110,1.897865,0.719250,1.544,4.154
1999-02,53.279,30.796485,20.012248,18.403355,25.512681,5.953575,44.015566,3.991446,16.895273,3.907415,...,3.1950,2.194475,2.078022,1.317705,0.938303,1.839885,1.854454,0.762141,1.524,4.536
1999-03,60.889,35.960414,23.959072,20.372880,30.371093,7.119918,47.614680,4.621626,20.008613,4.597218,...,3.9530,2.402243,2.457765,1.665029,1.012288,2.237680,2.302472,0.952522,1.998,5.367
1999-04,57.283,33.602412,20.801553,19.305764,26.656324,6.036068,44.728631,4.125728,16.999465,3.789916,...,3.8861,1.953003,2.164333,1.508982,0.936300,1.790408,2.127787,0.837487,2.034,6.017
1999-05,56.489,29.159320,20.835861,19.716198,24.496967,5.846184,42.431049,4.604527,16.677323,4.028047,...,4.0413,2.225595,2.235884,1.385499,0.891305,1.930022,2.219436,0.929694,2.220,4.711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,162.064,62.140888,36.649795,45.864572,51.113944,21.928952,134.229554,29.910008,71.594563,10.040161,...,19.2841,19.260493,30.712375,7.541088,4.346351,12.590424,20.338017,3.739214,5.898,31.787
2023-05,167.128,53.069122,38.472018,48.841326,55.606944,26.531895,142.529135,32.039593,79.841320,11.259351,...,21.7068,21.638910,32.536573,7.882918,4.401769,14.172895,21.905433,4.497862,6.262,37.443
2023-06,167.303,61.874647,39.368010,46.799481,60.938162,27.621806,150.471826,31.789015,80.262084,11.861299,...,20.6013,20.843931,32.486796,7.956390,4.475453,14.698290,22.805849,3.985981,5.415,34.289
2023-07,159.623,61.852347,39.025560,43.912860,54.392407,23.723906,139.267407,29.124319,74.991401,10.972422,...,20.8622,20.077953,29.253843,7.393661,4.716494,12.571060,18.912339,NaN,6.060,30.804


In [48]:
indicator = 'MEI' # Main Economic Indicators
subject = 'XTIMVA01' # Import - Value (goods)
measure = 'CXML' # US Dollars, monthly level
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Import Value')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.XTIMVA01.CXML.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2014-01

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,71.766,23.830091,24.435228,15.364684,25.144105,5.474216,36.785426,4.592146,16.481105,3.619162,...,1.7722,2.226510,3.117614,1.384903,2.352363,1.815298,1.906379,0.779530,1.906,2.525
1999-02,73.849,22.815911,24.621921,16.719499,24.796618,6.155598,37.829074,4.547039,16.044851,3.566755,...,1.8087,2.788102,3.210976,1.156381,2.453248,2.113624,1.945508,0.833150,1.856,2.773
1999-03,84.072,25.074027,28.072306,19.073719,28.746496,7.426152,41.489432,5.484792,18.774355,4.263362,...,1.9764,3.045149,4.044296,1.289481,3.229183,2.426659,2.565453,0.893484,2.078,3.259
1999-04,79.980,24.942305,25.432672,17.999164,25.263149,5.951536,38.701389,4.958185,15.982599,3.414864,...,2.1219,3.334086,3.628604,1.181464,2.400235,2.214675,2.237634,0.878240,1.873,3.092
1999-05,80.965,22.444231,24.178147,17.951649,24.897961,5.754935,38.979187,5.119372,16.036668,3.464770,...,1.9592,3.409307,3.698490,1.337550,2.496814,2.142580,2.238138,0.785570,1.931,2.723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,250.752,65.412176,56.461941,44.998257,64.587949,19.141536,119.422292,20.544640,70.924097,9.336482,...,15.3476,28.060944,29.998972,6.471050,6.716828,12.182306,18.515222,4.942585,6.091,24.404
2023-05,264.349,63.127962,62.695556,49.323522,67.736942,20.504346,127.815461,24.400257,72.994943,10.905459,...,21.2796,34.191875,31.029117,6.965880,7.610873,12.930068,19.971061,5.418513,7.386,26.070
2023-06,257.887,61.597132,61.445973,49.758131,70.585813,22.297460,126.440131,22.147282,71.450593,10.955629,...,17.1505,26.062334,30.351157,6.366087,7.288777,13.000198,19.800842,5.015212,7.248,25.252
2023-07,255.662,62.322096,58.762240,44.649932,65.212495,20.140515,119.321998,22.875983,69.795110,10.463509,...,19.5703,32.294928,28.337366,6.579927,7.430222,11.801046,18.230681,NaN,6.709,25.314


In [49]:
indicator = 'MEI' # Main Economic Indicators
subject = 'XTNTVA01' # Net Trade - Value (goods)
measure = 'CXML' # US Dollars, monthly level
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Net Trade Value')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.XTNTVA01.CXML.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel',
       'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'India',
       'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Indonesia', 'Turkiye', 'Poland', 'Chile', 'Greece', 'Hungary',
       'Czechia', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone'], dtype='object')

Missing first:
Saudi Arabia 2016-05

Missing last:

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,-19.330,6.663006,-5.146987,2.133514,-0.521865,0.272614,4.839436,-0.591879,0.997881,0.258876,...,1.2456,-0.314967,-1.108630,-0.016914,-1.461738,-0.148188,-0.008514,-0.060280,-0.362,1.629
1999-02,-20.570,7.980574,-4.609673,1.683856,0.716063,-0.202022,6.186492,-0.555593,0.850423,0.340660,...,1.3863,-0.542395,-1.132954,0.161324,-1.514945,-0.273739,-0.091055,-0.071009,-0.332,1.763
1999-03,-23.183,10.886387,-4.113234,1.299161,1.624597,-0.306234,6.125248,-0.863166,1.234258,0.333857,...,1.9766,-0.593000,-1.586531,0.375548,-2.216895,-0.188979,-0.262981,0.059038,-0.080,2.108
1999-04,-22.697,8.660107,-4.631119,1.306600,1.393174,0.084532,6.027242,-0.832457,1.016866,0.375053,...,1.7642,-1.341007,-1.464271,0.327518,-1.463935,-0.424267,-0.109846,-0.040753,0.161,2.925
1999-05,-24.476,6.715089,-3.342286,1.764549,-0.400994,0.091249,3.451862,-0.514845,0.640655,0.563277,...,2.0821,-1.149740,-1.462606,0.047949,-1.605508,-0.212557,-0.018702,0.144124,0.289,1.988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,-88.688,-3.271287,-19.812146,0.866315,-13.474005,2.787416,14.807262,9.365368,0.670466,0.703679,...,3.9365,-8.800451,0.713402,1.070038,-2.370478,0.408117,1.822796,-1.203371,-0.193,7.383
2023-05,-97.221,-10.058840,-24.223538,-0.482196,-12.129998,6.027548,14.713674,7.639336,6.846377,0.353892,...,0.4272,-12.552965,1.507456,0.917037,-3.209103,1.242827,1.934372,-0.920651,-1.124,11.373
2023-06,-90.584,0.277515,-22.077963,-2.958650,-9.647651,5.324346,24.031695,9.641734,8.811491,0.905670,...,3.4508,-5.218404,2.135639,1.590303,-2.813324,1.698092,3.005007,-1.029230,-1.833,9.037
2023-07,-96.039,-0.469749,-19.736680,-0.737072,-10.820088,3.583391,19.945408,6.248336,5.196291,0.508913,...,1.2919,-12.216975,0.916477,0.813734,-2.713728,0.770014,0.681658,NaN,-0.649,5.490


In [73]:
indicator = 'MEI' # Main Economic Indicators
subject = 'SLRTTO02' # Total retail sales value
measure = 'IXOBSA' # Index 2015=100 s.a.
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Retail Sales Value MoM')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.SLRTTO02.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'Austria',
       'Portugal', 'Eurozone', 'South Africa', 'Turkiye', 'Poland', 'Chile',
       'Greece', 'Hungary', 'Czechia', 'Colombia'],
      dtype='object')

No data:
Index(['Australia', 'New Zealand', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'Mexico', 'Indonesia', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Ireland 2000-01
Turkiye 2010-01
Chile 2005-01

Missing last:
Israel 2017-07
Poland 2018-12
Chile 2016-12
Colombia 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,56.803239,95.823656,60.878844,50.646098,57.495209,82.416242,85.2,NaN,88.909642,77.144748,...,NaN,NaN,30.490060,NaN,76.244896,37.110210,59.723454,25.498565,NaN,NaN
1999-02,57.366150,96.867280,60.646037,51.133003,58.795100,83.751724,85.2,NaN,88.594864,78.295657,...,NaN,NaN,31.705615,NaN,76.751506,36.790417,57.191856,26.498618,NaN,NaN
1999-03,57.662058,96.867280,60.995247,51.400114,58.895092,84.423822,86.4,NaN,90.107830,83.133105,...,NaN,NaN,34.485697,NaN,76.109734,38.751815,58.843541,25.926813,NaN,NaN
1999-04,58.060406,97.910904,60.878844,51.369252,58.395134,80.135708,84.2,NaN,90.128139,71.636809,...,NaN,NaN,33.816377,NaN,77.968294,38.922371,58.280610,25.702217,NaN,NaN
1999-05,58.633459,96.487780,61.577263,51.165638,59.295059,84.914360,84.2,NaN,90.564766,78.495632,...,NaN,NaN,35.803325,NaN,77.612035,38.105122,58.776651,26.047072,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03,150.687822,109.852831,134.096421,142.298544,139.088409,109.500000,136.2,NaN,138.723121,117.109759,...,NaN,941.6,NaN,NaN,123.700000,205.017561,152.312693,NaN,NaN,NaN
2023-04,151.328830,108.655525,136.540887,143.780178,137.888509,107.300000,137.2,NaN,139.223204,119.309942,...,NaN,979.5,NaN,NaN,126.200000,204.716508,150.712559,NaN,NaN,NaN
2023-05,152.127301,110.152158,137.588515,143.914480,137.588534,109.700000,140.2,NaN,141.023504,117.709809,...,NaN,1032.7,NaN,NaN,128.400000,202.107376,150.612551,NaN,NaN,NaN
2023-06,152.324573,109.453729,139.334562,144.046551,139.388384,111.400000,140.4,NaN,143.623937,119.309942,...,NaN,1082.5,NaN,NaN,125.100000,204.917210,151.812651,NaN,NaN,NaN


In [77]:
indicator = 'MEI' # Main Economic Indicators
subject = 'SLRTCR03' # Passenger Car Registration
measure = 'IXOB' # Index 2015=100
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Passenger Car Registration')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.SLRTCR03.IXOB.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'Switzerland',
       'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain',
       'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland',
       'New Zealand', 'Austria', 'Portugal', 'Eurozone', 'China', 'India',
       'Korea', 'Brazil', 'Turkiye', 'Poland', 'Greece', 'Hungary', 'Czechia',
       'Colombia'],
      dtype='object')

No data:
Index(['France', 'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia',
       'Chile', 'Argentina', 'Russia'],
      dtype='object')

Missing first:
Israel 2004-01
China 2001-01
India 2001-05
Poland 2003-01
Hungary 2003-01
Czechia 2003-01
Colombia 2003-01

Missing last:
Eurozone 2022-12
China 2018

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,86.630504,79.369935,82.859370,71.254854,NaN,86.314547,102.309701,82.657757,245.544008,60.246211,...,NaN,14.296435,NaN,NaN,491.119202,NaN,NaN,NaN,NaN,NaN
1999-02,106.299693,117.842473,38.294242,78.845803,NaN,73.633854,100.782211,98.953815,138.349171,70.024970,...,NaN,13.186813,NaN,NaN,284.787082,NaN,NaN,NaN,NaN,NaN
1999-03,124.964701,193.467074,168.624072,122.122299,NaN,131.348065,160.907187,125.919218,162.907311,99.140584,...,NaN,25.440901,NaN,NaN,373.452588,NaN,NaN,NaN,NaN,NaN
1999-04,116.771862,87.135702,83.408449,131.367556,NaN,108.701088,137.788588,103.584566,157.176366,76.557691,...,NaN,40.686143,NaN,NaN,393.620389,NaN,NaN,NaN,NaN,NaN
1999-05,131.707057,87.050954,80.150886,138.756349,NaN,103.418994,124.892687,105.550891,144.411261,76.917717,...,NaN,39.624765,NaN,NaN,366.883014,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04,45.871928,85.654163,60.599134,34.310887,NaN,65.798749,75.961700,35.349624,79.231779,70.495325,...,NaN,108.871616,120.834279,NaN,162.086433,132.935947,95.794366,NaN,NaN,NaN
2023-05,45.364577,79.780365,66.164648,40.513700,NaN,79.121363,92.437716,42.165439,88.487816,81.882585,...,NaN,139.587242,131.356413,NaN,205.113187,139.933395,105.509471,NaN,NaN,NaN
2023-06,44.431168,100.615089,80.774239,49.162598,NaN,92.487995,104.854147,47.464043,110.466229,107.305035,...,NaN,121.575985,141.592509,NaN,212.031027,145.546902,106.455511,NaN,NaN,NaN
2023-07,NaN,98.350407,65.580028,36.010681,NaN,NaN,91.056948,39.922200,76.609325,65.925324,...,NaN,154.382203,123.909194,NaN,195.979104,129.686022,83.625794,NaN,NaN,NaN


In [82]:
indicator = 'MEI' # Main Economic Indicators
subject = 'ODCNPI03' # Permits issued - Residential Buildings
measure = 'IXOBSA' # Index 2015=100
freq = 'M' # Monthly

df = get_oecd_data(indicator, subject, measure, freq, 
                   countries_before_subject=True,
                   file_name='Permits issued - Residential Buildings')
df

https://stats.oecd.org/SDMX-JSON/data/MEI/AUS+AUT+BEL+CAN+CHL+COL+CZE+DNK+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+JPN+KOR+MEX+NLD+NZL+NOR+POL+PRT+ESP+SWE+CHE+TUR+GBR+USA+ARG+BRA+CHN+IND+IDN+RUS+SAU+ZAF+EA19.ODCNPI03.IXOBSA.M/all?startTime=1999-01&contentType=csv
Available countries:
Index(['Canada', 'France', 'Germany', 'Australia', 'Netherlands', 'Sweden',
       'Spain', 'Finland', 'Belgium', 'Norway', 'Israel', 'New Zealand',
       'Portugal', 'Eurozone', 'Korea', 'South Africa', 'Turkiye', 'Poland',
       'Chile', 'Greece', 'Hungary', 'Colombia'],
      dtype='object')

No data:
Index(['United States', 'Japan', 'United Kingdom', 'Switzerland', 'Denmark',
       'Italy', 'Ireland', 'Austria', 'China', 'India', 'Brazil',
       'Saudi Arabia', 'Mexico', 'Indonesia', 'Czechia', 'Argentina',
       'Russia'],
      dtype='object')

Missing first:
Poland 2000-01

Missing last:
Netherlands 2018-12
Korea 2018-12
Turkiye 2018-12

Done


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Indonesia,Turkiye,Poland,Chile,Greece,Hungary,Czechia,Colombia,Argentina,Russia
1999-01,NaN,NaN,NaN,31.940123,136.969708,NaN,179.814985,64.235382,59.103118,NaN,...,NaN,71.554177,NaN,29.814500,1376.2,171.985668,NaN,23.994273,NaN,NaN
1999-02,NaN,NaN,NaN,35.274235,91.978798,NaN,161.413451,66.294144,46.807533,NaN,...,NaN,97.206968,NaN,31.305188,1300.9,176.085326,NaN,33.406692,NaN,NaN
1999-03,NaN,NaN,NaN,34.764994,81.197024,NaN,176.014668,64.427130,52.589508,NaN,...,NaN,49.683833,NaN,36.765404,1457.2,202.183151,NaN,29.712499,NaN,NaN
1999-04,NaN,NaN,NaN,37.320399,85.721869,NaN,176.214685,63.281691,61.580103,NaN,...,NaN,141.595463,NaN,23.181371,1309.2,225.381218,NaN,30.043077,NaN,NaN
1999-05,NaN,NaN,NaN,37.436800,85.802042,NaN,179.114926,61.510551,67.055638,NaN,...,NaN,68.700036,NaN,29.961002,1310.6,236.380302,NaN,17.978079,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03,NaN,NaN,NaN,148.045000,92.717343,NaN,92.707726,64.709948,NaN,NaN,...,NaN,NaN,118.759467,43.132099,600.4,166.186151,NaN,71.111378,NaN,NaN
2023-04,NaN,NaN,NaN,142.129616,104.217356,NaN,82.206851,61.431691,NaN,NaN,...,NaN,NaN,136.185402,46.143561,383.1,195.083743,NaN,61.380373,NaN,NaN
2023-05,NaN,NaN,NaN,159.922711,85.857798,NaN,83.806984,72.296802,NaN,NaN,...,NaN,NaN,112.887269,51.768837,517.8,205.082910,NaN,85.864572,NaN,NaN
2023-06,NaN,NaN,NaN,157.871036,99.624287,NaN,73.606134,66.654440,NaN,NaN,...,NaN,NaN,117.323193,46.834621,NaN,162.486459,NaN,64.139224,NaN,NaN


In [52]:
# Main indicators - Production
#PRMNTO01+PRINTO01+PRCNTO01.ST+STSA+IXOB+IXOBSA+ML+MLSA+QL+QLSA.Q+M/all?startTime=2016-Q1&endTime=2023-Q3